# Fine-Tuning LLM on Kaggle GPUs Using Unsloth
This notebook demonstrates how to leverage Kaggle GPUs and fine-tune a large language model using Unsloth and the LoRA (Low-Rank Adaptation) technique for classification tasks. To learn more about Unsloth, click [here](https://docs.unsloth.ai/).

You can find the inference notebook [here](https://www.kaggle.com/code/mehedi457/inference-qwen2-5-math-1-5b-unsloth).

**References:**
* https://www.kaggle.com/code/aleaiest/lb-0-945-qwen2-5-32b-gptq
* https://www.kaggle.com/code/cdeotte/gemma2-9b-it-cv-0-945
* https://www.kaggle.com/code/xbar19/jigsaw-llama3-1-8b-instruct-training

In [1]:
!pip install pip3-autoremove -q
!pip install -q torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu124
!pip install unsloth -q
!pip install --upgrade -q transformers==4.53.2 "huggingface_hub>=0.34.0" "datasets>=3.4.1,<4.0.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import pandas as pd
import numpy as np
import unsloth
import torch
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset, load_dataset
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import FastLanguageModel
from trl import SFTTrainer
from unsloth.chat_templates import get_chat_template

# Configuration
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
VER = 1
model_name ="unsloth/Qwen2.5-Math-1.5B-bnb-4bit"
EPOCHS = 1
DIR = f"ver_{VER}"
os.makedirs(DIR, exist_ok=True)
MAX_LEN = 2048

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-08-11 12:09:45.767707: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754914186.096494      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754914186.196524      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
# Load and preprocess the training data
train = pd.read_csv('/kaggle/input/map-charting-student-math-misunderstandings/train.csv')
train.Misconception = train.Misconception.fillna('NA')
train['target'] = train.Category + ":" + train.Misconception

# We use LabelEncoder to convert the 65 unique target strings into integers from 0 to 64.
le = LabelEncoder()
train['label'] = le.fit_transform(train['target'])
n_classes = len(le.classes_)
print(f"Train shape: {train.shape} with {n_classes} target classes")

Train shape: (36696, 9) with 65 target classes


In [4]:
# Feature Engineering
idx = train.apply(lambda row: row.Category.split('_')[0], axis=1) == 'True'
correct = train.loc[idx].copy()
correct['c'] = correct.groupby(['QuestionId', 'MC_Answer']).MC_Answer.transform('count')
correct = correct.sort_values('c', ascending=False)
correct = correct.drop_duplicates(['QuestionId'])
correct = correct[['QuestionId', 'MC_Answer']]
correct['is_correct'] = 1

train = train.merge(correct, on=['QuestionId', 'MC_Answer'], how='left')
train.is_correct = train.is_correct.fillna(0)
train['is_correct'] = train.apply(lambda x: "yes" if x['is_correct'] == 1 else "no", axis=1)

In [5]:
# Create a list of unique characters to represent each class. This is the core of the new strategy.
# The model will learn to output one of these characters.
special_character_list = [
    '■', '□', '▲', '△', '▼', '▽', '◆', '◇', '○', '●', '★', '☆', '♦', '♥', '♠', '♣',
    '§', '†', '‡', '※', '∞', '±', '≠', '≈', '√', '∑', '∏', '∆', 'Ω', 'μ', '∂', '→',
    '←', '↑', '↓', '↔', '↕', '〈', '〉', '『', '』', '│', '─', '┌', '┐', '└', '┘', '┼',
    '█', '▓', '▒', '£', '¥', '€', '₩', '©', '®', '™', '♪', '♫', '☀', '☁', '☂', '☃', '☎'
]

# Map the integer label to its corresponding special character
train['special_label'] = train['label'].apply(lambda x: special_character_list[int(x)])

In [6]:
# Load model and tokenizer using Unsloth
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=MAX_LEN,
    dtype=None,
    load_in_4bit=True,
)

# Prepare the model for LoRA fine-tuning
model = FastLanguageModel.get_peft_model(
    model,
    r=8, # Increased rank for better learning
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=64,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=42,
    use_rslora=False,
    loftq_config=None,
)

==((====))==  Unsloth 2025.8.4: Fast Qwen2 patching. Transformers: 4.53.2.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


<string>:36: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.


model.safetensors:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/165 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Unsloth 2025.8.4 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [7]:
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen2.5", # Using standard qwen2 chat template
)

# This detailed prompt explicitly tells the model the classification task and the exact output format.
# It provides the mapping from the special character to the full class name.
class_mappings = [f"{special_character_list[i]}: {le.classes_[i]}" for i in range(n_classes)]

In [8]:
SYS_PROMPT = f"""You are an expert at analyzing math student responses. Your task is to classify the student's explanation into one of the following Category:Misconception classes.

Respond with ONLY the single character corresponding to the correct classification.

Available classifications:
{', '.join(class_mappings)}

Analyze the given input and provide your classification.
"""

# Create user prompt
user_prompt_template = """Question: {QuestionText}
Answer: {MC_Answer}
Correct? {CorrectFlag}
Student Explanation: {StudentExplanation}
"""

dataset_chat = [
    [
        {"role": "system", "content": SYS_PROMPT},
        {
            "role": "user",
            "content": user_prompt_template.format(
                QuestionText=row["QuestionText"],
                MC_Answer=row["MC_Answer"],
                CorrectFlag=row["is_correct"],
                StudentExplanation=row["StudentExplanation"],
            )
        },
        {
            "role": "assistant",
            "content": row["special_label"]
        }
    ]
    for _, row in train.iterrows()
]

# Convert to tokenized dataset
def formatting_prompts_func(dataset):
    # This function is now simplified as we handle the chat template application directly.
    texts = [
        tokenizer.apply_chat_template(
            ex, tokenize=False, add_generation_prompt=False
        ) for ex in dataset
    ]
    return {"text": texts}

dataset_tokenized = formatting_prompts_func(dataset_chat)

In [9]:
dataset_tokenized = Dataset.from_dict(dataset_tokenized)

In [10]:
dataset_tokenized[0]

{'text': "<|im_start|>system\nYou are an expert at analyzing math student responses. Your task is to classify the student's explanation into one of the following Category:Misconception classes.\n\nRespond with ONLY the single character corresponding to the correct classification.\n\nAvailable classifications:\n■: False_Correct:NA, □: False_Misconception:Adding_across, ▲: False_Misconception:Adding_terms, △: False_Misconception:Additive, ▼: False_Misconception:Base_rate, ▽: False_Misconception:Certainty, ◆: False_Misconception:Definition, ◇: False_Misconception:Denominator-only_change, ○: False_Misconception:Division, ●: False_Misconception:Duplication, ★: False_Misconception:Firstterm, ☆: False_Misconception:FlipChange, ♦: False_Misconception:Ignores_zeroes, ♥: False_Misconception:Incomplete, ♠: False_Misconception:Incorrect_equivalent_fraction_addition, ♣: False_Misconception:Interior, §: False_Misconception:Inverse_operation, †: False_Misconception:Inversion, ‡: False_Misconception:I

In [11]:
# Training Arguments with Unsloth optimizations
from unsloth import is_bfloat16_supported

training_args = TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    warmup_ratio=0.05,
    # num_train_epochs=EPOCHS,
    learning_rate=2e-4,
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=1,
    optim="paged_adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=42,
    output_dir=f"./{DIR}",
    # save_strategy="epoch",
    save_strategy="steps",
    save_steps=0.10,
    max_steps = 50,
    save_total_limit=5,
    report_to="none",
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset_tokenized,
    dataset_text_field="text",
    max_seq_length=MAX_LEN,
    dataset_num_proc=2,
    packing=False, # Important for classification tasks
    args=training_args,
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/36696 [00:00<?, ? examples/s]

In [12]:
from unsloth.chat_templates import train_on_responses_only

trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|im_start|>user\n",
    response_part = "<|im_start|>assistant\n",
)

Map (num_proc=4):   0%|          | 0/36696 [00:00<?, ? examples/s]

In [13]:
# Train the model
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 36,696 | Num Epochs = 1 | Total steps = 50
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 9,232,384 of 1,552,946,688 (0.59% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,9.174100
2,9.191200
3,8.977900
4,8.843800
5,7.683600
6,7.308900
7,6.074800
8,4.572800
9,5.469500
10,5.431400
